In [1]:
import socket

In [2]:
so = socket.socket(type=socket.SOCK_DGRAM)

In [3]:
so.connect(('127.0.0.1', 1234))  # 调用了connect send方法将向这个地址发送

In [4]:
so.send(b'hello\r\n')

7

In [5]:
so.recv(1024)

b'127.0.0.1:38999\nhello'

In [6]:
s = socket.socket(type=socket.SOCK_DGRAM)

In [13]:
s.sendto(b'hello\r\n', ('127.0.0.1', 1234)) # 如果没有调用connect， 只能用sendto

OSError: [Errno 9] Bad file descriptor

In [8]:
s.recv(1024)

b'127.0.0.1:46262\nhello'

In [9]:
s.fileno()

54

In [10]:
so.fileno()

53

In [11]:
s.close()

In [12]:
so.close()

**无论有没有调用connect都需要close**

**close的作用是释放文件描述符**

In [14]:
import threading

In [24]:
class ChatClient:
    def __init__(self, host='127.0.0.1', port=1234):
        self.sock = socket.socket(type=socket.SOCK_DGRAM)
        self.host = host
        self.port = port
        self.event = threading.Event()
    
    def heartbeat(self):
        while not self.event.wait(5):
            self.sock.sendto(b'#ping#', (self.host, self.port))
    
    def recv(self):
        while not self.event.is_set():
            message = self.sock.recv(1024)
            print(message.strip())
    
    def start(self):
        threading.Thread(target=self.heartbeat, name='heartbeat', daemon=True).start()
        threading.Thread(target=self.recv, name='recv', daemon=True).start()
        while not self.event.is_set():
            message = input('{}:{}>>'.format(self.host, self.port))
            message = message.strip()
            if message == '/exit':
                self.sock.close()
                self.event.set()
                return
            self.sock.sendto(message.encode(), (self.host, self.port))
                
        

In [25]:
ChatClient(port=1235).start()

127.0.0.1:1235>>/exit


In [28]:
with socket.socket() as so:
    so.connect(('127.0.0.1', 1234))
    so.send(b'hello\r\n')
    print(so.recv(1024))

b'hello'
